In [ ]:
#====================================Assigning Heads to Bodies Algorithm==============================================
import numpy as np
import math

# Put the Yolov8 Annotation matrix (from labels folder) for the selected input image:
predict = np.array([
    [0, 0.521875, 0.4921875, 0.4921875, 0.26015625],
    [0, 0.64765625, 0.7046875, 0.365625, 0.31015625],
    [1, 0.33515625, 0.5390625, 0.1234375, 0.16640625],
    [1, 0.534375, 0.65625, 0.134375, 0.20859375],
    [0, 0.9140625, 0.24375, 0.171875, 0.3046875]
])

# Function to calculate the top-left and bottom-right corners of a bounding box
def calculate_corners(center, size, width, height):
    center_x, center_y = center
    box_width, box_height = size
    top_left = (int(center_x * width - box_width * width // 2), int(center_y * height - box_height * height // 2))
    bottom_right = (int(center_x * width + box_width * width // 2), int(center_y * height + box_height * height // 2))
    return top_left, bottom_right

# Function to check overlap between two bounding boxes
def check_overlap(top_left1, bottom_right1, top_left2, bottom_right2):
    x1_min, y1_min = top_left1
    x1_max, y1_max = bottom_right1
    x2_min, y2_min = top_left2
    x2_max, y2_max = bottom_right2
    return (x1_min < x2_max and x1_max > x2_min and y1_min < y2_max and y1_max > y2_min)

# Function to calculate Euclidean distance between two centers
def calculate_distance(center1, center2):
    return math.sqrt((center1[0] - center2[0]) ** 2 + (center1[1] - center2[1]) ** 2)

# Separate bodies and heads
bodies = predict[predict[:, 0] == 0]
heads = predict[predict[:, 0] == 1]

# Initialize lists to track assigned heads
assigned_heads = []

# Image dimensions (for normalization)
image_width, image_height = 640, 640

# List to store body-head pairs
body_head_pairs = []

# Iterate over each body to find the corresponding head
for i, body in enumerate(bodies):
    body_center = (body[1], body[2])
    body_size = (body[3], body[4])
    body_top_left, body_bottom_right = calculate_corners(body_center, body_size, image_width, image_height)

    best_head = None
    best_distance = float('inf')

    for j, head in enumerate(heads):
        if j in assigned_heads:
            continue  # Skip already assigned heads

        head_center = (head[1], head[2])
        head_size = (head[3], head[4])
        head_top_left, head_bottom_right = calculate_corners(head_center, head_size, image_width, image_height)

        if check_overlap(body_top_left, body_bottom_right, head_top_left, head_bottom_right):
            best_head = head
            assigned_heads.append(j)
            break

        distance = calculate_distance(body_center, head_center)
        if distance < best_distance:
            best_distance = distance
            best_head = head

    if best_head is not None:
        body_head_pairs.append({
            'body': body,
            'head': best_head
        })
    else:
        body_head_pairs.append({
            'body': body,
            'head': None  # No head assigned
        })

# Output the body-head pairs
for pair in body_head_pairs:
    body = pair['body']
    head = pair['head']
    print(f"Body Coordinates: Center ({body[1]}, {body[2]}), Size ({body[3]}, {body[4]})")
    if head is not None:
        print(f"Head Coordinates: Center ({head[1]}, {head[2]}), Size ({head[3]}, {head[4]})")
    else:
        print("No head detected for this body.")
    print("--------")


In [ ]:
#=========================AffectingHeads+DrawBBs+DrawLines+CalDist+CalAngle(H1-H2)(B1-B2)============================

import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math

def initialize_plot(image):
    """Initialize the plot with the given image."""
    fig, ax = plt.subplots(1, figsize=(10, 8))
    ax.imshow(image)
    ax.set_xlim(0, image.shape[1])
    ax.set_ylim(image.shape[0], 0)  # Invert y-axis to match image coordinates
    return fig, ax

def calculate_corners(center, size, width, height):
    """Calculate the top-left and bottom-right corners of a bounding box."""
    center_x, center_y = center
    box_width, box_height = size
    top_left = (int(center_x * width - box_width * width // 2), int(center_y * height - box_height * height // 2))
    bottom_right = (int(center_x * width + box_width * width // 2), int(center_y * height + box_height * height // 2))
    return top_left, bottom_right

def draw_bounding_box(ax, top_left, bottom_right, bbox_color, label=None):
    """Draw a bounding box on the given axis."""
    width = bottom_right[0] - top_left[0]
    height = bottom_right[1] - top_left[1]
    rect = patches.Rectangle(top_left, width, height, linewidth=2, edgecolor=bbox_color, facecolor='none')
    ax.add_patch(rect)
    if label is not None:
        ax.text(top_left[0], top_left[1] - 10, label, color=bbox_color, fontsize=12, weight='bold')

def check_overlap(top_left1, bottom_right1, top_left2, bottom_right2):
    """Check if two bounding boxes overlap."""
    x1_min, y1_min = top_left1
    x1_max, y1_max = bottom_right1
    x2_min, y2_min = top_left2
    x2_max, y2_max = bottom_right2
    return (x1_min < x2_max and x1_max > x2_min and y1_min < y2_max and y1_max > y2_min)

def calculate_distance(center1, center2):
    """Calculate Euclidean distance between two centers."""
    return math.sqrt((center1[0] - center2[0]) ** 2 + (center1[1] - center2[1]) ** 2)

def draw_lines_and_calculate_distance(ax, center1, size1, center2, size2, line_color='r', bbox_color1='cyan', bbox_color2='magenta', pixels_per_cm_x=1, pixels_per_cm_y=1):
    """Draw lines and calculate distance between bounding boxes."""
    height, width, _ = image.shape
    top_left1, bottom_right1 = calculate_corners(center1, size1, width, height)
    top_left2, bottom_right2 = calculate_corners(center2, size2, width, height)
    draw_bounding_box(ax, top_left1, bottom_right1, bbox_color1)
    draw_bounding_box(ax, top_left2, bottom_right2, bbox_color2)
    center1_scaled = (int(center1[0] * width), int(center1[1] * height))
    center2_scaled = (int(center2[0] * width), int(center2[1] * height))
    distance_pixels = math.sqrt((center1_scaled[0] - center2_scaled[0]) ** 2 + (center1_scaled[1] - center2_scaled[1]) ** 2)
    distance_cm_x = distance_pixels / pixels_per_cm_x
    distance_cm_y = distance_pixels / pixels_per_cm_y
    distance_cm = (distance_cm_x + distance_cm_y) / 2
    ax.plot([center1_scaled[0], center2_scaled[0]], [center1_scaled[1], center2_scaled[1]], line_color, linewidth=2)
    return distance_cm

def calculate_angle_between_lines(p1, p2, p3, p4):
    """Calculate the angle between two lines."""
    def vector_from_points(p1, p2):
        return np.array([p2[0] - p1[0], p2[1] - p1[1]])

    v1 = vector_from_points(p1, p2)
    v2 = vector_from_points(p3, p4)

    cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    angle = np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))
    return angle

# Load the selected image (from images folder) and get pixel dimensions
image = cv2.imread("/content/img1605_png.rf.fb24d91e31bfc66a496777d212392e9d.jpg")
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_pixel_height, image_pixel_width, _ = image.shape

# Given printed image default dimensions in centimeters
printed_width_cm = 16.7
printed_height_cm = 16.7

# Calculate Pixels Per Centimeter
pixels_per_cm_x = image_pixel_width / printed_width_cm
pixels_per_cm_y = image_pixel_height / printed_height_cm

# Initialize the plot with the original image
fig, ax = initialize_plot(image_rgb)

# YoloV8 Annotation matrix of the image
predict = np.array([
    [0, 0.43125, 0.2828125, 0.5390625, 0.2421875],
    [0, 0.14921875, 0.42421875, 0.225, 0.546875],
    [1, 0.1828125, 0.23671875, 0.1578125, 0.16640625],
    [1, 0.21015625, 0.3296875, 0.1, 0.15390625]
])

# Separate bodies and heads
bodies = predict[predict[:, 0] == 0]
heads = predict[predict[:, 0] == 1]

# Initialize lists to track assigned heads
assigned_heads = []

# List to store body-head pairs
body_head_pairs = []
body_ids = np.arange(1, len(bodies) + 1)
head_ids = np.arange(1, len(heads) + 1)

# Create a dictionary for head ID lookup
head_id_dict = {tuple(head[1:]): head_id for head, head_id in zip(heads, head_ids)}

# Iterate over each body to find the corresponding head
for i, body in enumerate(bodies):
    body_center = (body[1], body[2])
    body_size = (body[3], body[4])
    body_top_left, body_bottom_right = calculate_corners(body_center, body_size, image_pixel_width, image_pixel_height)

    best_head = None
    best_distance = float('inf')

    for j, head in enumerate(heads):
        if j in assigned_heads:
            continue  # Skip already assigned heads

        head_center = (head[1], head[2])
        head_size = (head[3], head[4])
        head_top_left, head_bottom_right = calculate_corners(head_center, head_size, image_pixel_width, image_pixel_height)

        if check_overlap(body_top_left, body_bottom_right, head_top_left, head_bottom_right):
            best_head = head
            assigned_heads.append(j)
            break

        distance = calculate_distance(body_center, head_center)
        if distance < best_distance:
            best_distance = distance
            best_head = head

    if best_head is not None:
        # Use tuple of the head coordinates to find the ID
        head_id = head_id_dict.get(tuple(best_head[1:]), None)
        body_head_pairs.append({
            'body': body,
            'head': best_head,
            'body_id': body_ids[i],
            'head_id': head_id
        })
    else:
        body_head_pairs.append({
            'body': body,
            'head': None,  # No head assigned
            'body_id': body_ids[i]
        })

# Draw bounding boxes for all bodies
for pair in body_head_pairs:
    body = pair['body']
    body_id = pair['body_id']

    # Draw body bounding box
    body_center = (body[1], body[2])
    body_size = (body[3], body[4])
    body_top_left, body_bottom_right = calculate_corners(body_center, body_size, image_pixel_width, image_pixel_height)
    draw_bounding_box(ax, body_top_left, body_bottom_right, 'blue', label=f'Body {body_id}')

    if pair['head'] is not None:
        head = pair['head']
        head_id = pair.get('head_id')  # Use .get() to avoid KeyError

        # Draw head bounding box
        head_center = (head[1], head[2])
        head_size = (head[3], head[4])
        head_top_left, head_bottom_right = calculate_corners(head_center, head_size, image_pixel_width, image_pixel_height)
        draw_bounding_box(ax, head_top_left, head_bottom_right, 'red', label=f'Head {head_id}')

    else:
        # No head assigned
        ax.text(body_top_left[0], body_top_left[1] - 10, 'No Head', color='white', fontsize=12, weight='bold')

# Draw lines and calculate distances and angles
distances_head = []
distances_body = []
angles = []

for i, pair1 in enumerate(body_head_pairs):
    body1 = pair1['body']
    head1 = pair1.get('head')

    if head1 is not None:
        body1_center = (body1[1], body1[2])
        body1_size = (body1[3], body1[4])
        body1_top_left, body1_bottom_right = calculate_corners(body1_center, body1_size, image_pixel_width, image_pixel_height)
        head1_center = (head1[1], head1[2])
        head1_size = (head1[3], head1[4])
        head1_top_left, head1_bottom_right = calculate_corners(head1_center, head1_size, image_pixel_width, image_pixel_height)

        for pair2 in body_head_pairs[i+1:]:
            body2 = pair2['body']
            head2 = pair2.get('head')

            if head2 is not None:
                body2_center = (body2[1], body2[2])
                body2_size = (body2[3], body2[4])
                body2_top_left, body2_bottom_right = calculate_corners(body2_center, body2_size, image_pixel_width, image_pixel_height)
                head2_center = (head2[1], head2[2])
                head2_size = (head2[3], head2[4])
                head2_top_left, head2_bottom_right = calculate_corners(head2_center, head2_size, image_pixel_width, image_pixel_height)

                if check_overlap(body1_top_left, body1_bottom_right, body2_top_left, body2_bottom_right):
                    # Draw line between heads
                    distance_cm = draw_lines_and_calculate_distance(ax, head1_center, head1_size, head2_center, head2_size, 'y', 'red', 'red', pixels_per_cm_x, pixels_per_cm_y)
                    distances_head.append(f'dist H{pair1["head_id"]}-H{pair2["head_id"]}={distance_cm:.2f} cm')

                    # Draw line between bodies
                    distance_cm = draw_lines_and_calculate_distance(ax, body1_center, body1_size, body2_center, body2_size, 'g', 'blue', 'blue', pixels_per_cm_x, pixels_per_cm_y)
                    distances_body.append(f'dist B{pair1["body_id"]}-B{pair2["body_id"]}={distance_cm:.2f} cm')

                    # Calculate angle between lines
                    angle = calculate_angle_between_lines(
                        (head1_center[0], head1_center[1]),
                        (head2_center[0], head2_center[1]),
                        (body1_center[0], body1_center[1]),
                        (body2_center[0], body2_center[1])
                    )
                    angles.append(f'angle of cross(H{pair1["head_id"]}-H{pair2["head_id"]} with B{pair1["body_id"]}-B{pair2["body_id"]})={angle:.2f}°')

# Print all distances
print(" ; ".join(distances_head))
print(" ; ".join(distances_body))
print(" ; ".join(angles))

# Remove axis and save the image with original dimensions
ax.axis('off')
fig.set_size_inches(image_rgb.shape[1] / 100, image_rgb.shape[0] / 100)
plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
plt.savefig('/content/resFinal_no_axis.jpg', bbox_inches='tight', pad_inches=0)
plt.show()

In [ ]:
#==========================================Features Extraction=======================================================
import numpy as np

# Set the print options for NumPy to display numbers with two decimal places
np.set_printoptions(formatter={'float': '{: 0.2f}'.format})

# Initialize a list to store feature vectors
features = []

# Iterate over each overlapping pair and create the feature vector
for i in range(len(body_head_pairs)):
    for j in range(i + 1, len(body_head_pairs)):
        body1 = body_head_pairs[i]['body']
        body2 = body_head_pairs[j]['body']

        body1_center = (body1[1], body1[2])
        body1_size = (body1[3], body1[4])
        body2_center = (body2[1], body2[2])
        body2_size = (body2[3], body2[4])
        body1_top_left, body1_bottom_right = calculate_corners(body1_center, body1_size, image_pixel_width, image_pixel_height)
        body2_top_left, body2_bottom_right = calculate_corners(body2_center, body2_size, image_pixel_width, image_pixel_height)

        if check_overlap(body1_top_left, body1_bottom_right, body2_top_left, body2_bottom_right):
            head1 = next((pair['head'] for pair in body_head_pairs if np.array_equal(pair['body'], body1)), None)
            head2 = next((pair['head'] for pair in body_head_pairs if np.array_equal(pair['body'], body2)), None)

            nbHeads = sum([1 if head1 is not None else 0, 1 if head2 is not None else 0])

            dist_H1_H2 = None
            if head1 is not None and head2 is not None:
                head1_center = (head1[1], head1[2])
                head2_center = (head2[1], head2[2])
                head1_size = (head1[3], head1[4])
                head2_size = (head2[3], head2[4])
                dist_H1_H2 = draw_lines_and_calculate_distance(ax, head1_center, head1_size, head2_center, head2_size, 'r', 'red', 'red', pixels_per_cm_x, pixels_per_cm_y)
                dist_H1_H2 = round(dist_H1_H2, 2) if dist_H1_H2 is not None else None

            dist_B1_B2 = draw_lines_and_calculate_distance(ax, body1_center, body1_size, body2_center, body2_size, 'g', 'blue', 'blue', pixels_per_cm_x, pixels_per_cm_y)
            dist_B1_B2 = round(dist_B1_B2, 2)

            angle = None
            if head1 is not None and head2 is not None:
                angle = calculate_angle_between_lines(head1_center, head2_center, body1_center, body2_center)
                angle = round(angle, 2) if angle is not None else None

            # Create the feature vector
            feature = [nbHeads, dist_H1_H2, dist_B1_B2, angle]

            # Combine the IDs of the overlapping pairs
            feature_id = f'{body_head_pairs[i]["body_id"]}_{body_head_pairs[j]["body_id"]}'

            # Add the feature vector to the list with its combined ID
            features.append((feature_id, feature))

# Optionally, print the feature vectors
if len(features) == 0:
    print("No overlap detected")
else:
    for feature_id, feature in features:
        print(f'feature{feature_id} = {feature}')

# Convert the list of features to a matrix (2D NumPy array)
if len(features) > 0:
    frameFeatures = np.array([feature for _, feature in features])
    print("\nframeFeatures matrix:")
    print(frameFeatures)
else:
    frameFeatures = np.empty((0, 4))  # Create an empty matrix with 4 columns if no features
    print("No feature vectors generated; frameFeatures matrix is empty.")


In [ ]:
#=====================================================Sexual Activity Classification Using Decision Tree Algorithm==================================================================
import numpy as np

def classify_overlap(frameFeatures):
    # Define reference values for mean and variance
    mean_dist_H1_H2 = 2.2
    var_dist_H1_H2 = 0.37

    mean_angle_mounting = 91
    var_angle_mounting = 4783.11
    mean_dist2_H1_H2 = 4.9
    var_dist2_H1_H2 = 0.23

    mean_dist3_H1_H2 = 4.37
    var_dist3_H1_H2 = 1.37
    mean_dist3_B1_B2 = 4.5
    var_dist3_B1_B2 = 0.6

    mean_dist4_H1_H2 = 7.4
    var_dist4_H1_H2 = 1.33
    mean_dist4_B1_B2 = 5.93
    var_dist4_B1_B2 = 1.28

    classifications = []

    for feature in frameFeatures:
        nbHeads, dist_H1_H2, dist_B1_B2, angle = feature

        if nbHeads < 2:
            classification = "Front Climb"
        else:
            # Calculate standard deviations from the reference means and variances
            sd_dist_H1_H2 = np.abs(dist_H1_H2 - mean_dist_H1_H2) / var_dist_H1_H2 if angle is not None else np.inf

            sd_angle_mounting = np.abs(angle - mean_angle_mounting) / var_angle_mounting if angle is not None else np.inf
            sd_dist2_H1_H2 = np.abs(dist_H1_H2 - mean_dist2_H1_H2) / var_dist2_H1_H2 if angle is not None else np.inf

            sd_dist3_H1_H2 = np.abs(dist_H1_H2 - mean_dist3_H1_H2) / var_dist3_H1_H2 if angle is not None else np.inf
            sd_dist3_B1_B2 = np.abs(dist_B1_B2 - mean_dist3_B1_B2) / var_dist3_B1_B2 if angle is not None else np.inf

            sd_dist4_H1_H2 = np.abs(dist_H1_H2 - mean_dist4_H1_H2) / var_dist4_H1_H2 if angle is not None else np.inf
            sd_dist4_B1_B2 = np.abs(dist_B1_B2 - mean_dist4_B1_B2) / var_dist4_B1_B2 if angle is not None else np.inf

            if sd_dist_H1_H2 <= 2.5:  # Assuming 2.5 as the threshold for "near"
                classification = "HeadToHead"
            elif sd_angle_mounting <= 2.5 and sd_dist2_H1_H2 <= 2.5:
                classification = "Mounting"
            elif sd_dist3_H1_H2 <= 2.5 and sd_dist3_B1_B2 <= 2.5:
                classification = "HeadToBody"
            elif sd_dist4_H1_H2 <= 2.5 and sd_dist4_B1_B2 <= 2.5:
                classification = "Sniffing"

        classifications.append(classification)

    return classifications

if len(features) > 0:
    frameFeatures = np.array([feature for _, feature in features])
    print("\nframeFeatures matrix:")
    print(frameFeatures)

    # Apply the classification function
    classifications = classify_overlap(frameFeatures)

    # Print the classifications
    for i, classification in enumerate(classifications):
        print(f'Feature row {i+1} classification: {classification}')
else:
    print("No feature vectors generated; frameFeatures matrix is empty.")
